In [ ]:
__author__ = "Jithin Pradeep"
__copyright__ = "Copyright (C) 2018 Jithin Pradeep"
__license__ = "MIT License"
__version__ = "1.0"

In [ ]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import os
import sys
import tarfile
from IPython.display import display, Image
from scipy import ndimage
from sklearn.linear_model import LogisticRegression
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

import pandas as pd
from sklearn.cross_validation import train_test_split

import tensorflow as tf
import tensorflow.contrib.layers as layers
# Config the matplotlib backend as plotting inline in IPython
%matplotlib inline

In [ ]:
from skimage.color import rgb2gray
from scipy.misc import imread, imsave, imresize
from skimage import data, io, filters
from skimage.filters import threshold_otsu
image_width = 20
image_height = 20

#Function for one hot code encoding implementation
def dense_to_one_hot(labels_dense, num_classes):
    num_labels = len(labels_dense)
    labels_one_hot = np.zeros((num_labels, num_classes))
    index=0
    for item in labels_dense:
        labels_one_hot[index,item] = 1
        #print(item)
        #print(index,item)
        index = index + 1
    #print(len(labels_one_hot))
    #print(labels_one_hot.shape)
    #print(len(labels_one_hot[0]))
    return labels_one_hot 

#Pre processing and loading the iamge dataset
def loaddata(path, train):
    extension = "Bmp"
    image_size = 28  # Pixel width and height.
    pixel_depth = 255.0  # Number of levels per pixel.
    image_files = os.listdir(os.getcwd()+'/'+path+'/')
    image_dataset = np.zeros((len(image_files), image_width, image_height))
    
    if train:
        train_data = pd.read_csv(os.getcwd()+'/data/trainLabels.csv')
        labels_flat = train_data[[1]].values.ravel()
        image_ids =  train_data[[0]].values.ravel()
        item = 0
        for image_id in image_ids:
            image_file = '' +str(image_id)+'.'+extension
            #print (os.getcwd()+'/'+path+'/'+image_file)
            image_data = (ndimage.imread(os.getcwd()+'/'+path+'/'+image_file).astype(float)- 
                          pixel_depth / 2) / pixel_depth
            #resizing
            image_data = imresize(image_data,(image_width,image_height))
            #converting to grayscale
            image_data = rgb2gray(image_data)
            #displaying the last training dataset image 
            plt.imshow(image_data)
            
            image_dataset[item]=image_data
            item = item + 1
     
        image_dataset = image_dataset.reshape([-1, image_width, image_height, 1])

        image_dataset = np.multiply(image_dataset, 1.0 / pixel_depth)
        #print(len(image_dataset))
        #print(image_dataset.shape)
        #print(len(image_dataset[0]))
        #print(os.getcwd()+'/'+path+'/', image_dataset[0].shape, len(image_dataset))
        
        
        # preprocessing the training image label and preforming one hot code encoding 
        labels_flat_list = []
        label_flat_conv = []
        for item in labels_flat :
            labels_flat_list.append(item) 
            label_flat_conv.append(item) 

        labels_count = np.unique(labels_flat).shape[0]
        
        maplabel = []
        for item in np.unique(labels_flat) :
            maplabel.append(item)  
            
        for item in label_flat_conv :
            pos=label_flat_conv.index(item)
            label_flat_conv[pos]= maplabel.index(item)  
            
        labels = dense_to_one_hot(np.array(label_flat_conv), labels_count)
        labels = labels.astype(np.uint8)

        
        return image_dataset, labels
    #LOding the test dataset 
    item = 0
    for image_file in image_files:

        image_data = (ndimage.imread(os.getcwd()+'/'+path+'/'+image_file).astype(float)- 
                          pixel_depth / 2) / pixel_depth
            
        image_data = imresize(image_data,(image_width,image_height))
        
        image_data = rgb2gray(image_data)  
        image_dataset[item] = image_data
        item = item + 1
     
    image_dataset = image_dataset.reshape([-1, image_width, image_height, 1])

    image_dataset = np.multiply(image_dataset, 1.0 / pixel_depth)
    #print(len(image_dataset))
    #print(image_dataset.shape)
    #print(len(image_dataset[0]))
    #print(os.getcwd()+'/'+path+'/', image_dataset[0].shape, len(image_dataset))
        
    return image_dataset    
        
        

In [ ]:
#Model CNN for First step with Julia
class TFConvNet(object):
    def __init__(self, feature_num, class_num, is_training, epochs=500000, batch_size=100, learning_rate=5e-4):
        self.batch_size = batch_size
        self.epochs = epochs
        self.weight_decay = 1e-3

        self.bn_params = {
            # Decay for the moving averages.
            'decay': 0.999,
            'center': True,
            'scale': True,
            # epsilon to prevent 0s in variance.
            'epsilon': 0.001,
            # None to force the updates during train_op
            'updates_collections': None,
            'is_training': is_training
        }

        self.feature_num = feature_num
        self.class_num = class_num

        self.keep_prob = tf.placeholder(tf.float32)
        self.X = tf.placeholder(tf.float32, [None, feature_num])
        self.y_ = tf.placeholder(tf.float32, [None, class_num])

        with tf.contrib.framework.arg_scope(
                [layers.convolution2d],
                kernel_size=3, stride=1, padding='SAME', activation_fn=tf.nn.elu,
                normalizer_fn=layers.batch_norm, normalizer_params=self.bn_params,
                weights_initializer=layers.variance_scaling_initializer(),
                weights_regularizer=layers.l2_regularizer(self.weight_decay)
        ):
            self.X = tf.reshape(self.X, [-1, image_width, image_height, 1])

            net = layers.convolution2d(self.X, num_outputs=128)
            net = layers.convolution2d(net, num_outputs=128)
            net = layers.max_pool2d(net, kernel_size=2)
            net = layers.relu(net, num_outputs=128)

            net = layers.convolution2d(net, num_outputs=256)
            net = layers.convolution2d(net, num_outputs=256)
            net = layers.max_pool2d(net, kernel_size=2)
            net = layers.relu(net, num_outputs=256)
            
            net = layers.convolution2d(net, num_outputs=512)
            net = layers.convolution2d(net, num_outputs=512)
            net = layers.max_pool2d(net, kernel_size=2)
            net = layers.relu(net, num_outputs=512)
            
            net = layers.dropout(net, keep_prob=self.keep_prob)
            net = layers.relu(net, num_outputs=32)

            net = layers.flatten(net, [-1, 7 * 7 * 32])
            net = layers.fully_connected(net, num_outputs=4096, activation_fn=tf.nn.relu)
            net = layers.dropout(net, keep_prob=self.keep_prob)
            net = layers.fully_connected(net, num_outputs=4096, activation_fn=tf.nn.relu)
            net = layers.dropout(net, keep_prob=self.keep_prob)

            net = layers.fully_connected(net, num_outputs=self.class_num, activation_fn=tf.nn.relu)
            self.y = layers.softmax(net)

        #Adam optimizer and cross entropy loss function 
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(self.y, self.y_))
        self.optimizer = tf.train.AdamOptimizer(learning_rate).minimize(self.loss) 

        
        pred = tf.equal(tf.argmax(self.y, 1), tf.argmax(self.y_, 1))
        self.acc = tf.reduce_mean(tf.cast(pred, tf.float32))

        self.sess = tf.Session()
        
    #training the model    
    def train(self, X_train, y_train, X_val, y_val, keep_prob=0.5):
        print("\nStarting to train\n")
        self.sess.run(tf.global_variables_initializer())

        batch_start = 0
        batch_end = batch_start + self.batch_size
        for iteration in range(self.epochs):
            _, loss, probs = self.sess.run(
                [self.optimizer, self.loss, self.y],
                feed_dict={
                    self.X: X_train[batch_start:batch_end],
                    self.y_: y_train[batch_start:batch_end],
                    self.keep_prob: keep_prob
                }
            )

            if iteration % 50 == 0:
                train_acc = self.sess.run(self.acc, feed_dict={
                    self.X: X_train[batch_start:batch_end],
                    self.y_: y_train[batch_start:batch_end],
                    self.keep_prob: 1.0
                })

                val_acc = self.sess.run(
                    self.acc,
                    feed_dict={self.X: X_val, self.y_: y_val, self.keep_prob: 1.0}
                )

                print('Iteration: {}, loss: {:2.4}, train acc: {:.3%}, validation acc: {:.3%}'.format(
                    iteration, loss, train_acc, val_acc))

                if val_acc >= 0.991:
                    print('Validation acc good!!! Breaking the run!!!')
                    break

            batch_start = batch_end
            batch_end += self.batch_size

            if batch_end > len(X_train):
                batch_start = 0
                batch_end = batch_start + self.batch_size

        print("\nTraining ended")





In [ ]:
train_x, train_y = loaddata('data/train',True)
test_x = loaddata('data/test',False)

In [ ]:
# Ensuring the loaded images and one hot encoding are properly assigned.
print('labels({0[0]},{0[1]})'.format(train_y.shape))
print ('labels[{0}] => {1}'.format(1,train_y[6282]))

In [ ]:
image = train_x[2]
print('image shape:', train_x.shape)
train_x.shape
plt.imshow(image.reshape(image_width,image_height))


In [ ]:
train_x, val_x, train_y, val_y = train_test_split(train_x, train_y, test_size=0.2)
print('Train size:', len(train_x))
print('Validation size:', len(val_x))

In [ ]:
#Let's start ...go...go..
conv_net = TFConvNet(feature_num=(image_width*image_height), class_num=62, is_training=False)
conv_net.train(train_x, train_y, val_x, val_y, keep_prob=0.5)
# Note : Submission file creation code has been removed.